In [2]:
from transformers import (
    AutoProcessor, Qwen2VLForConditionalGeneration
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")
model = Qwen2VLForConditionalGeneration.from_pretrained(
"Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
from datasets import load_dataset

ds = load_dataset("nielsr/countbench")

train_data = ds["train"]

# Convert the first 10 examples to a Pandas DataFrame
df = pd.DataFrame(train_data)

# Show the DataFrame
print(df)

                                             image_url  \
0    http://www.wirerealm.com/wp-content/uploads/20...   
1    https://wmich.edu/sites/default/files/styles/7...   
2    http://img0.etsystatic.com/000/0/5304297/il_fu...   
3    https://cdn.vectorstock.com/i/thumb-large/74/3...   
4    https://laurelleaffarm.com/item-photos/vintage...   
..                                                 ...   
535  http://cdn.images.express.co.uk/img/dynamic/78...   
536  https://www.lvsdecorativearts.co.uk/upload/ima...   
537  https://a.1stdibscdn.com/archivesD/upload/8511...   
538  http://3.bp.blogspot.com/_dWmIOlGB7_0/TA6LK-d7...   
539  http://tse2.mm.bing.net/th?id=OIP.2Bfd_b6EPJqU...   

                                                  text  number  \
0    We review the ten best gaming headsets in the ...      10   
1                background photo of three light bulbs       3   
2    City prints: Set of three big prints - $150.00...       3   
3         Set of eight arrows in all di

In [24]:
from io import BytesIO

img = df.loc[0]["image"]   # keep it as PIL image

buffer = BytesIO()
img.save(buffer, format="JPEG")
image_bytes = buffer.getvalue()

prompt = "Describe the 15 headsets in the image." 

In [26]:
from general_utils import *
from knockout_utils import * 
model_name = "qwen"

task = "color" #ignore this param

inputs = build_inputs(task, image_bytes, model_name, processor, prompt, model, device="cuda")


In [27]:
predicted_answer = generate_text(model, processor, inputs, model_name, max_new_tokens=50, do_sample=False, temperature=0.0)
print(predicted_answer)

/oscar/data/ceickhof/mgolovan/llava/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/oscar/data/ceickhof/mgolovan/llava/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/oscar/data/ceickhof/mgolovan/llava/lib64/python3.9/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


The image shows a collection of 15 different headsets, each with unique designs and features. Here is a detailed description of each headset:

1. **Top Row, Left to Right:**
   - **Headset 1:** Black and


In [28]:
heads = {(0, 3), (11, 15), (0, 6)}
max_new_tokens = 50
do_sample = False
temperature = 0.0
zeroer = HeadMeanAblation(model, model_name)


with zeroer.knockout(heads): 
    ablated = generate_text(model, processor, inputs, model_name, max_new_tokens, do_sample, temperature)
    print(ablated)

The image shows a collection of 10 different gaming headsets. Here is a description of each headset from left to right, top to bottom:

1. **Top Left**: A black and red headset with a microphone.
2. **Top Middle
